In [1]:
# Dependencies
from splinter import Browser
from requests import get
import re
import pandas as pd
import numpy as np
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import select
import urllib.request
from bs4 import BeautifulSoup
import requests
import pymongo

In [2]:
# r = requests.get(url)
# soup = BeautifulSoup(r.text, 'html.parser')
# print(soup)
executable_path = {"executable_path": "chromedriver.exe"}
url = 'https://twitchtracker.com/channels/viewership'

In [3]:
driver = webdriver.Chrome()
driver.get(url)

In [4]:
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')
# print(soup)

In [5]:
streamer_table = soup.find_all(
                'table', 
                {'id': 'channels'})
print(streamer_table)

[<table class="table table-condensed text-center" id="channels">
<thead>
<tr>
<th></th>
<th></th>
<th></th>
<th class="sortable active" data-original-title="Last 30 days" data-toggle="tooltip" title="" width="8%"><a class="color-viewers" href="/channels/viewership">AVG Viewers</a></th>
<th data-original-title="Last 30 days" data-toggle="tooltip" title="" width="8%"><span class="color-streamed">Time Streamed</span></th>
<th class="sortable" data-original-title="Overall" data-toggle="tooltip" title="" width="10%"><a class="color-viewersMax" href="/channels/peak-viewers">All Time Peak Viewers</a></th>
<th class="sortable" data-original-title="Last 30 days" data-toggle="tooltip" title="" width="9%"><a class="color-watched" href="/channels/hours-watched">Hours Watched</a></th>
<th class="sortable" data-original-title="Last 30 days" data-toggle="tooltip" title="" width="7%"><i class="fas fa-crown color-gold"></i><br/><a class="color-gold" href="/channels/ranking">Rank</a></th>
<th class="sor

In [6]:
# Empty lists to store data
streamer_handle = []
avg_views = []
time_streamed = []
peak_viewers= []
hours_watched = []
rank_value = []
total_views = []

In [7]:
# Parse through each tag and pull out text info
channel = soup.find_all('a', {'style' : 'color:inherit'})
views = soup.find_all('td', {'class' : 'color-viewers'})
time = soup.find_all('td', {'class' : 'color-streamed'})
peak_views = soup.find_all('td', {'class' : 'color-viewersMax'})
hours = soup.find_all('td', {'class' : 'color-watched'})
rank = soup.find_all('span', {'class' : 'rank-value'})
all_views = soup.find_all('td', {'class' : 'color-views'})


In [8]:
for streamer in rank:
    rank_value.append(streamer.get_text())
print(rank_value)

['1', '2', '9', '11', '10', '5', '7', '4', '3', '20', '12', '21', '22', '16', '19', '18', '15', '27', '6', '37']


In [9]:
for streamer in channel:
    streamer_handle.append(streamer.get_text())
print(streamer_handle)

['tommyinnit', 'auronplay', 'QuackityHQ', 'karljacobs', 'BLASTPremier', 'ibai', 'Rubius', 'TheGrefg', 'xQcOW', 'GeorgeNotFound', 'LCK_Korea', 'LEC', 'WilburSoot', 'LCK', 'winstrike_tv', 'LCS', 'Tubbo', 'RiotGamesBrazil', 'GamesDoneQuick', 'Sapnap']


In [10]:
for streamer in all_views:
    total_views.append(streamer.get_text())
print(total_views)

['30.8M', '99.6M', '13.5M', '9.25M', '61.8M', '139M', '136M', '107M', '265M', '8.12M', '217M', '92.1M', '9.38M', '99.0M', '22.0M', '114M', '23.2M', '188M', '348M', '4.29M']


In [11]:
for streamer in views:
    avg_views.append(streamer.get_text())
print(avg_views)


['248,185', '160,756', '113,096', '107,280', '88,628', '85,383', '83,170', '79,455', '78,445', '74,280', '70,930', '66,650', '65,363', '64,950', '61,037', '59,328', '59,198', '55,956', '55,787', '55,230']


In [12]:
for streamer in time:
    time_streamed.append(streamer.get_text())

print(time_streamed)

['24hours', '89.9hours', '36.4hours', '24hours', '55.3hours', '111.6hours', '101.4hours', '166.9hours', '340.4hours', '16.1hours', '70.8hours', '41.5hours', '17.5hours', '65.7hours', '51.9hours', '71.4hours', '90hours', '43.2hours', '247.1hours', '10.9hours']


In [13]:
for streamer in peak_views:
    peak_viewers.append(streamer.get_text())
print(peak_viewers)

['650,237', '313,347', '232,299', '206,847', '208,744', '533,239', '344,676', '2,470,347', '222,720', '114,009', '365,983', '400,312', '215,211', '212,049', '235,473', '400,429', '203,070', '264,955', '247,432', '94,007']


In [14]:
for streamer in hours: 
    hours_watched.append(streamer.get_text())
print(hours_watched)

['5.96M', '14.5M', '4.11M', '2.58M', '4.90M', '9.53M', '8.43M', '13.3M', '26.7M', '1.19M', '5.02M', '2.77M', '1.15M', '4.27M', '3.16M', '4.24M', '5.32M', '2.42M', '13.8M', '601K']


In [15]:
tracker_df = {'Streamer Handle' : streamer_handle,
              'Average Viewers':avg_views,
              'Time Streamed (hrs)': time_streamed,
              'All Time Peak': peak_viewers, 
              'Hours Watched': hours_watched,
              'Rank' : rank_value, 
              'Total Views (million)' : total_views}
# tracker_df


In [16]:
# Put the combined lists into a dataframe
complete_tracker_df = pd.DataFrame.from_dict(tracker_df, orient='index')
complete_tracker_df = complete_tracker_df.transpose()

complete_tracker_df

,Streamer Handle,Average Viewers,Time Streamed (hrs),All Time Peak,Hours Watched,Rank,Total Views (million)
0,tommyinnit,"248,185",24hours,"650,237",5.96M,1,30.8M
1,auronplay,"160,756",89.9hours,"313,347",14.5M,2,99.6M
2,QuackityHQ,"113,096",36.4hours,"232,299",4.11M,9,13.5M
3,karljacobs,"107,280",24hours,"206,847",2.58M,11,9.25M
4,BLASTPremier,"88,628",55.3hours,"208,744",4.90M,10,61.8M
5,ibai,"85,383",111.6hours,"533,239",9.53M,5,139M
6,Rubius,"83,170",101.4hours,"344,676",8.43M,7,136M
7,TheGrefg,"79,455",166.9hours,"2,470,347",13.3M,4,107M
8,xQcOW,"78,445",340.4hours,"222,720",26.7M,3,265M
9,GeorgeNotFound,"74,280",16.1hours,"114,009",1.19M,20,8.12M


In [ ]:
# Cleaning up the DataFrame
